In [1]:
import pandas as pd

In [2]:
# Ler o arquivo Excel
df_grouped = pd.read_excel('agrupando.xlsx')

# Converter todos os valores da coluna 'box' para strings e preencher NaN com uma string vazia
df_grouped['box'] = df_grouped['box'].fillna('').astype(str)

# Agrupar por 'razao social' e concatenar 'box'
df_grouped = df_grouped.groupby(['matricula','nome'])['box'].apply(lambda x: ' '.join(x)).reset_index()

df_grouped


,matricula,nome,box
0,100004.0,LUIZ CARLOS DALL'AGNOL,Pav:D Box:72
1,100010.0,ANGELO AUGUSTO WELTER,Pav:E Box:41
2,100037.0,YOSHIHIRO TAKEUCHI,Pav:I Box:47
3,100041.0,TADASHI SADA,Pav:C Box:60
4,100171.0,SUNAO MAEDA,Pav:E Box:52
...,...,...,...
781,7100801.0,GRUPO VIZ. ZUCATTI,Pav:H Box:22
782,7100802.0,GRUPO VIZ. TRIVELIN,Pav:F Box:56
783,7500017.0,BANCO DO ESTADO DO RIO GRANDE DO SUL S/A,Pav:CUP Box:14 Pav:CUP Box:15 Pav:CUP Box:16
784,7500018.0,BANCO DO BRASIL S/A,Pav:CUP Box:10 Pav:CUP Box:10 Pav:CUP Box:12


In [3]:
import pandas as pd
import re

# Lendo o arquivo Excel
siger = pd.read_excel('siger.xlsx')

# Ajustando os valores na coluna 'box' para o formato desejado com regex
def format_box_value(x):
	if not isinstance(x, str):
		x = str(x)
	pattern = re.compile(r'(Pav:\w+ Box:\d+)')
	matches = pattern.findall(x)
	if matches:
		return matches
	else:
		return ['']

siger = siger.set_index(['matricula'])['box'].apply(format_box_value).explode().reset_index(name='box')

# Exibindo o resultado
print(siger)


      matricula             box
0     7500019.0    Pav:B2 Box:1
1     7500018.0  Pav:CUP Box:10
2     7500018.0  Pav:CUP Box:11
3     7500018.0  Pav:CUP Box:12
4     7500017.0  Pav:CUP Box:14
...         ...             ...
1179        NaN                
1180        NaN                
1181        NaN                
1182        NaN                
1183        NaN                

[1184 rows x 2 columns]


In [5]:
import pandas as pd
import re

# Lendo o arquivo Excel
teste = pd.read_excel('teste.xlsx')

# Ajustando os valores na coluna 'box' para o formato desejado com regex
def format_box_value(x):
	if not isinstance(x, str):
		x = str(x)
	pattern = re.compile(r'(Pav:\w+ Box:\d+)')
	matches = pattern.findall(x)
	if matches:
		return matches
	else:
		return ['Formato Incorreto']

# Ajustando os valores na coluna 'box'
teste = teste.set_index(['matricula'])['box'].apply(format_box_value).explode().reset_index(name='box')

# Exibindo o resultado
print(teste)


     matricula                box
0          NaN  Formato Incorreto
1    7500019.0  Formato Incorreto
2          NaN  Formato Incorreto
3    7500018.0     Pav:CUP Box:10
4    7500018.0     Pav:CUP Box:10
..         ...                ...
792   113863.0      Pav:SOM Box:8
793   113863.0      Pav:SOM Box:9
794        NaN  Formato Incorreto
795   100176.0       Pav:A8 Box:6
796   100176.0       Pav:A8 Box:7

[797 rows x 2 columns]


In [7]:
df_merged = pd.merge(teste, siger, on=['matricula', 'box'], how='outer', indicator=True)
df_merged['_merge'] = df_merged['_merge'].replace({'right_only': 'siger', 'left_only': 'atualizados', 'both': 'iguais'})

# Filtrar as diferenças
diferencas = df_merged[df_merged['_merge'] != 'both']


C:\Users\divtec-dados\AppData\Local\Temp\ipykernel_504\3725066816.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_merged['_merge'] = df_merged['_merge'].replace({'right_only': 'siger', 'left_only': 'atualizados', 'both': 'iguais'})


In [8]:
diferencas

,matricula,box,_merge
0,100003.0,,siger
1,100010.0,Pav:B2 Box:41,siger
2,100037.0,Pav:B2 Box:47,siger
3,100041.0,Pav:B2 Box:60,siger
4,100171.0,Pav:B2 Box:52,siger
...,...,...,...
1661,NaN,Pav:SOM Box:8,siger
1662,NaN,Pav:SOM Box:9,siger
1663,NaN,Pav:TC Box:32,siger
1664,NaN,Pav:TC Box:33,siger


In [9]:
diferencas.to_excel('Verificação SIGER 2.xlsx', index=False)